In [1]:
import tensorflow as tf
tf.config.experimental.set_memory_growth(
    tf.config.list_physical_devices('GPU')[0], True)
print(tf.__version__)
print("Usage of gpu: {}".format(tf.config.list_physical_devices()))
print("Built with CUDA: {}".format(tf.test.is_built_with_cuda()))
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2.5.0
Usage of gpu: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Built with CUDA: True
Num GPUs Available:  1


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

import os
from distutils.dir_util import copy_tree, remove_tree

from PIL import Image
from random import randint

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef as MCC
from sklearn.metrics import balanced_accuracy_score as BAS
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow_addons as tfa
import tensorflow.keras
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG
from tensorflow.keras.layers import SeparableConv2D, BatchNormalization, MaxPool2D
from sklearn.model_selection import KFold 
import pydot

print("TensorFlow Version:", tf.__version__)

TensorFlow Version: 2.5.0


In [3]:
base_dir = "C://Users/sarvi/Desktop/project/Alzheimers-Disease-Classification/ADNI5//"
root_dir = "./"
test_dir = base_dir + "test/"
train_dir = base_dir + "train/"
work_dir = root_dir + "dataset/"
save_dir = root_dir + "saved_models/"

In [4]:
IMG_SIZE = 256
IMAGE_SIZE = [256, 256]
DIM = (IMG_SIZE, IMG_SIZE)
ZOOM = [.99, 1.01]
BRIGHT_RANGE = [0.8, 1.2]
HORZ_FLIP = True
FILL_MODE = "constant"
DATA_FORMAT = "channels_last"

In [5]:
work_dr = IDG(rescale = 1./255, brightness_range=BRIGHT_RANGE, zoom_range=ZOOM, data_format=DATA_FORMAT, fill_mode=FILL_MODE, horizontal_flip=HORZ_FLIP)

train_data_gen = work_dr.flow_from_directory(directory=work_dir, target_size=DIM, batch_size=6500, shuffle=False)

Found 2614 images belonging to 5 classes.


In [6]:
train_data, train_labels = train_data_gen.next()
print(train_data.shape, train_labels.shape)
#Performing over-sampling of the data, since the classes are imbalanced
sm = SMOTE(random_state=42)
train_data, train_labels = sm.fit_resample(train_data.reshape(-1, IMG_SIZE * IMG_SIZE * 3), train_labels)
train_data = train_data.reshape(-1, IMG_SIZE, IMG_SIZE, 3)
print(train_data.shape, train_labels.shape)

(2614, 256, 256, 3) (2614, 5)
(5800, 256, 256, 3) (5800, 5)


In [7]:
train_data, test_data, train_labels, test_labels = train_test_split(train_data, train_labels, test_size = 0.2, random_state=42)
print(train_data.shape, test_data.shape, train_labels.shape, test_labels.shape)

(4640, 256, 256, 3) (1160, 256, 256, 3) (4640, 5) (1160, 5)


In [8]:
def construct_model(act='relu'):
    """Constructing a Sequential CNN architecture for performing the classification task. """
    
    model = Sequential([
        Input(shape=(*IMAGE_SIZE, 3)),
        Conv2D(16, 3, activation=act, padding='same'),
        MaxPool2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'),
        BatchNormalization(),
        Dropout(0.2),
        Conv2D(32, 3, activation=act, padding='same'),
        MaxPool2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'),
        BatchNormalization(),
        Dropout(0.2),
        Conv2D(64, 3, activation=act, padding='same'),
        MaxPool2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'),
        BatchNormalization(),
        Dropout(0.2),
        Conv2D(128, 3, activation=act, padding='same'),
        MaxPool2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'),
        BatchNormalization(),
        Dropout(0.2),
        Conv2D(256, 3, activation=act, padding='same'),
        BatchNormalization(),
        Dropout(0.2),
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.7),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(5, activation='softmax')        
    ], name = "cnn_model")

    return model

In [9]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
EPOCHS = 100
batch_size = 32
idg = IDG()
# k = 20
# kf = KFold(n_splits=k, random_state=None)
fold_var = 1

# for train_index, val_index in kf.split(train_data, train_labels):
X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, test_size = 0.2, random_state=42)
#     X_train, X_test = train_data[train_index, :], train_data[val_index, :]
#     y_train, y_test = train_labels[train_index], train_labels[val_index]
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

train_gen = idg.flow(X_train, y_train, batch_size=32, shuffle=True )
val_gen = idg.flow(X_test, y_test, batch_size=8, shuffle=True)



model = construct_model()
model.summary()

METRICS = [tf.keras.metrics.CategoricalAccuracy(name='acc'),
           tf.keras.metrics.AUC(name='auc'),
           tfa.metrics.F1Score(num_classes=5)]

model.compile(optimizer='adam',
              loss=tf.losses.CategoricalCrossentropy(),
              metrics=METRICS)

def get_model_name(k):
    return 'model_'+str(k)+'.h5'

checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var),
                                                monitor='val_acc', verbose=1,
                                                save_best_only=True, mode='max')

my_callback = checkpoint
CALLBACKS = [my_callback]
# EarlyStopping callback to make sure model is always learning
early_stopping = EarlyStopping(monitor='val_loss', patience=2)

(3712, 256, 256, 3) (928, 256, 256, 3) (3712, 5) (928, 5)
Model: "cnn_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 16)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 128, 16)      64        
_________________________________________________________________
dropout (Dropout)            (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________

In [10]:
del train_data
del test_data

In [11]:
history = model.fit(train_gen,
                   steps_per_epoch = int(3712 // batch_size),
                   epochs = EPOCHS,
                   verbose = 1,
                   validation_data = val_gen,
                   validation_steps = int(928 // batch_size),
                   callbacks = CALLBACKS
                   )

model.load_weights("/saved_models/model_"+str(fold_var)+".h5")

results = model.evaluate(val_gen)
results = dict(zip(model.metrics_names, results))

VALIDATION_ACCURACY.append(results['accuracy'])
VALIDATION_LOSS.append(results['loss'])

tf.keras.backend.clear_session()

Epoch 1/100
 25/116 [=====>........................] - ETA: 36s - loss: 2.2505 - acc: 0.2138 - auc: 0.5194 - f1_score: 0.2134

KeyboardInterrupt: 

In [10]:
# # With K-Fold
# VALIDATION_ACCURACY = []
# VALIDAITON_LOSS = []
# EPOCHS = 100
# batch_size = 32
# idg = IDG()
# k = 20
# kf = KFold(n_splits=k, random_state=None)
# fold_var = 1

# for train_index, val_index in kf.split(train_data, train_labels):
#     # X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, test_size = 0.2, random_state=42)
#     X_train, X_test = train_data[train_index, :], train_data[val_index, :]
#     y_train, y_test = train_labels[train_index], train_labels[val_index]
#     print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

#     train_gen = idg.flow(X_train, y_train, batch_size=32, shuffle=True )
#     val_gen = idg.flow(X_test, y_test, batch_size=8, shuffle=True)



#     model = construct_model()
#     model.summary()

#     METRICS = [tf.keras.metrics.CategoricalAccuracy(name='acc'),
#                tf.keras.metrics.AUC(name='auc'),
#                tfa.metrics.F1Score(num_classes=5)]

#     model.compile(optimizer='adam',
#                   loss=tf.losses.CategoricalCrossentropy(),
#                   metrics=METRICS)

#     def get_model_name(k):
#         return 'model_'+str(k)+'.h5'

#     checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var),
#                                                     monitor='val_acc', verbose=1,
#                                                     save_best_only=True, mode='max')

#     my_callback = checkpoint
#     CALLBACKS = [my_callback]
#     # EarlyStopping callback to make sure model is always learning
#     early_stopping = EarlyStopping(monitor='val_loss', patience=2)

#     del train_data
#     del test_data

#     history = model.fit(train_gen,
#                        steps_per_epoch = int(3712 // batch_size),
#                        epochs = EPOCHS,
#                        verbose = 1,
#                        validation_data = val_gen,
#                        validation_steps = int(928 // batch_size),
#                        callbacks = CALLBACKS
#                        )

#     model.load_weights("/saved_models/model_"+str(fold_var)+".h5")

#     results = model.evaluate(val_gen)
#     results = dict(zip(model.metrics_names, results))

#     VALIDATION_ACCURACY.append(results['accuracy'])
#     VALIDATION_LOSS.append(results['loss'])

#     tf.keras.backend.clear_session()

#     fold_var += 1

(3712, 256, 256, 3) (928, 256, 256, 3) (3712, 5) (928, 5)
Model: "cnn_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 16)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 128, 16)      64        
_________________________________________________________________
dropout (Dropout)            (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________

In [ ]:
# tf.keras.backend.clear_session()
# from numba import cuda 
# device = cuda.get_current_device()
# device.reset()